In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
!ollama pull llama3
!pip install ollama
import ollama

In [ ]:
#HSE

# # Файл с именами файлов, которые уже обработаны
# PROCESSED_DIR = "/kaggle/input/dataset-real-diploma/processed-json-hse"
# # Директория с json после парсера и обработки
# INPUT_DIR = "/kaggle/input/dataset-real-diploma/dataset-hse/dataset-hse"
# # Куда сохранять результат
# OUTPUT_DIR = "./processed"
# EXCLUDE_KEYS = {'заголовок', 'год', 'тема', 'код_темы'}

In [ ]:
# SPBU

# Файл с именами файлов, которые уже обработаны
PROCESSED_DIR = "/kaggle/input/dataset-real-diploma/processed-json-hse"
# Директория с json после парсера и обработки
INPUT_DIR = "/kaggle/input/dataset-real-diploma/dataset-spbu/dataset-spbu"
# Куда сохранять результат
OUTPUT_DIR = "./processed"
EXCLUDE_KEYS = {'id','заголовок', 'год', 'тема'}

In [ ]:
! ollama pull qwen2.5vl:7b

In [8]:
import json
def load_json_list(json_path: str):
    """
    Читает json-файл и возвращает соответствующий python-список.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    if not isinstance(data, list):
        raise ValueError(f"Ожидался список, а в файле {json_path} — {type(data).__name__}")
    return data

data_processed = load_json_list(PROCESSED_DIR)

In [ ]:
import glob
import json
import os
import re

prompt = "Продолжи следующий текст до целого абзаца: "

def get_first_sentence(text: str) -> str:
    """
    Берём первое предложение из текста.
    """
    match = re.match(r"(.+?[\.!?])", text.strip(), flags=re.DOTALL)
    return match.group(1) if match else text.strip()

def generate_text_for_prompt(prompt: str, sentence) -> str:
    """
    Генерируем текст по промпту.
    """
    response = ollama.chat(model='qwen2.5vl:7b', messages=[
  {
    'role': 'user',
    'content': prompt + '"' + sentence + '"',
  },
    ])
    return(response['message']['content'])


def process_file(path: str):
    with open(path, "r", encoding="utf-8") as rf:
        data = json.load(rf)

    new_data = {}
    # Сначала копируем «исключённые» ключи
    for key in EXCLUDE_KEYS:
        if key in data:
            new_data[key] = data[key]

    # Обрабатываем все остальные ключи
    for key, value in data.items():
        if key in EXCLUDE_KEYS:
            continue
        if not isinstance(value, list):
            new_data[key] = value
            continue

        new_list = []
        for real_text in value:
            sentence = get_first_sentence(real_text)

            generated = generate_text_for_prompt(prompt, sentence)

            new_list.append({
                "real_text": real_text,
                "generated_text": generated
            })

        new_data[key] = new_list

    # Сохраняем результат
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    fname = os.path.basename(path)
    out_path = os.path.join(OUTPUT_DIR, fname)
    with open(out_path, "w", encoding="utf-8") as wf:
        json.dump(new_data, wf, ensure_ascii=False, indent=2)

def main():
    json_paths = glob.glob(os.path.join(INPUT_DIR, "*.json"))
    for p in json_paths:
        fname = os.path.basename(p) 
        if fname in data_processed:
            print (f"Пропускаем {fname}, уже обработан")
            continue
        print(f"Обрабатываем {p} ...")
        process_file(p)
    print("Готово. Все файлы в", OUTPUT_DIR)

if __name__ == "__main__":
    main()
